In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt


For this question I first trained a model on MNIST. I used tensorflow. Then, I did hyperparameter tuning. To use the methods from class, I reduced the dataset by half randomly and again trained the model. Finally, I reduced the dataset by half randomly and did hyperparameter tuning and trained the model. I calculated the compression ratios for all of them.

# 1) Original

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images, test_images = train_images / 255.0, test_images / 255.0

train_images = train_images[..., tf.newaxis]
test_images = test_images[..., tf.newaxis]


In [3]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])


In [4]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [5]:
history = model.fit(train_images, train_labels, epochs=10, 
                    validation_data=(test_images, test_labels))


Epoch 1/10
1875/1875 [==============================] - 14s 7ms/step - loss: 0.1491 - accuracy: 0.9541 - val_loss: 0.0548 - val_accuracy: 0.9826
Epoch 2/10
1875/1875 [==============================] - 13s 7ms/step - loss: 0.0473 - accuracy: 0.9855 - val_loss: 0.0312 - val_accuracy: 0.9896
Epoch 3/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0345 - accuracy: 0.9893 - val_loss: 0.0289 - val_accuracy: 0.9909
Epoch 4/10
1875/1875 [==============================] - 22s 12ms/step - loss: 0.0253 - accuracy: 0.9922 - val_loss: 0.0316 - val_accuracy: 0.9899
Epoch 5/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0212 - accuracy: 0.9935 - val_loss: 0.0379 - val_accuracy: 0.9876
Epoch 6/10
1875/1875 [==============================] - 21s 11ms/step - loss: 0.0163 - accuracy: 0.9947 - val_loss: 0.0292 - val_accuracy: 0.9908
Epoch 7/10
1875/1875 [==============================] - 23s 12ms/step - loss: 0.0130 - accuracy: 0.9959 - val_loss: 0.0279 - v

In [6]:
test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)
print('\nTest accuracy:', test_acc)


313/313 - 1s - loss: 0.0347 - accuracy: 0.9908 - 668ms/epoch - 2ms/step

Test accuracy: 0.9908000230789185


In [23]:
input_dim = (28, 28, 1)

X_train=train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
X_train=X_train / 255.0
X_test = test_images.reshape(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1)
X_test=X_test/255.0

In [55]:
import numpy as np

t_size = np.prod(input_dim)
t_params = model.count_params()
compression_ratio = t_params / t_size
print("The original dataset compression ratio is:", compression_ratio)
print()

input_size = np.prod(input_dim)
output_size = np.prod(model.layers[-8].output_shape[1:])  
t_params = model.count_params()
compression_ratio = output_size / input_size
print("The original dataset compression ratio after last layer:", compression_ratio)
print()

compression_layer_ratios = []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        input_size = np.prod(layer.input_shape[1:])
        output_size = np.prod(layer.output_shape[1:])
        compression_ratio_layer = output_size / input_size
        compression_layer_ratios.append(compression_ratio_layer)

total_cr = np.prod(compression_ratio_layer)
print("The original dataset total compression ratio is:", total_cr)
print(compression_layer_ratios)
print()
for i, ratio in enumerate(compression_layer_ratios):
    print(f"Compression Ratio for Conv2D layer of Original Dataset {i+1}: {ratio:.2f}")

The original dataset compression ratio is: 119.03316326530613

The original dataset compression ratio after last layer: 27.591836734693878

The original dataset total compression ratio is: 0.36
[27.591836734693878, 1.4319526627218935, 0.36]

Compression Ratio for Conv2D layer of Original Dataset 1: 27.59
Compression Ratio for Conv2D layer of Original Dataset 2: 1.43
Compression Ratio for Conv2D layer of Original Dataset 3: 0.36


## 2) Now changing hyperparameters:

In [32]:
model_modified = models.Sequential([
    layers.Conv2D(64, (3, 3), activation='relu', input_shape=(28, 28, 1)),  
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'), 
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),  
    layers.Flatten(),
    layers.Dense(128, activation='relu'),  
    layers.Dense(10)
])


model_modified.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                       loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                       metrics=['accuracy'])

history_modified = model_modified.fit(train_images, train_labels, epochs=15,  # Increased epochs
                                      validation_data=(test_images, test_labels))

test_loss_modified, test_acc_modified = model_modified.evaluate(test_images, test_labels, verbose=2)
print('\nModified Test accuracy:', test_acc_modified)


Epoch 1/15
1875/1875 [==============================] - 22s 12ms/step - loss: 0.1134 - accuracy: 0.9644 - val_loss: 0.0415 - val_accuracy: 0.9866
Epoch 2/15
1875/1875 [==============================] - 30s 16ms/step - loss: 0.0397 - accuracy: 0.9875 - val_loss: 0.0289 - val_accuracy: 0.9906
Epoch 3/15
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0275 - accuracy: 0.9915 - val_loss: 0.0323 - val_accuracy: 0.9895
Epoch 4/15
1875/1875 [==============================] - 40s 21ms/step - loss: 0.0211 - accuracy: 0.9932 - val_loss: 0.0241 - val_accuracy: 0.9919
Epoch 5/15
1875/1875 [==============================] - 41s 22ms/step - loss: 0.0157 - accuracy: 0.9950 - val_loss: 0.0328 - val_accuracy: 0.9911
Epoch 6/15
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0135 - accuracy: 0.9958 - val_loss: 0.0242 - val_accuracy: 0.9932
Epoch 7/15
1875/1875 [==============================] - 43s 23ms/step - loss: 0.0119 - accuracy: 0.9962 - val_loss: 0.0273 -

In [56]:
input_dim = (28, 28, 1)

X_train=train_images.reshape(train_images.shape[0], train_images.shape[1], train_images.shape[2], 1)
X_train=X_train / 255.0
X_test = test_images.reshape(test_images.shape[0], test_images.shape[1], test_images.shape[2], 1)
X_test=X_test/255.0

t_size = np.prod(input_dim)
t_params = model_modified.count_params()
compression_ratio = t_params / t_size
print("The hyperparameter tuning compression ratio is:", compression_ratio)
print()

input_size = np.prod(input_dim)
output_size = np.prod(model_modified.layers[-8].output_shape[1:])  
t_params = model.count_params()
compression_ratio = output_size / input_size
print("The hyperparameter tuning compression ratio after last layer:", compression_ratio)
print()

compression_layer_ratios = []
for layer in model_modified.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        input_size = np.prod(layer.input_shape[1:])
        output_size = np.prod(layer.output_shape[1:])
        compression_ratio_layer = output_size / input_size
        compression_layer_ratios.append(compression_ratio_layer)

total_cr = np.prod(compression_ratio_layer)
print("The hyperparameter tuning total compression ratio is:", total_cr)
print(compression_layer_ratios)
print()
for i, ratio in enumerate(compression_layer_ratios):
    print(f"The hyperparameter tuning compression ratio for Conv2D layer {i+1}: {ratio:.2f}")

The hyperparameter tuning compression ratio is: 473.15561224489795

The hyperparameter tuning compression ratio after last layer: 55.183673469387756

The hyperparameter tuning total compression ratio is: 0.36
[55.183673469387756, 1.4319526627218935, 0.36]

The hyperparameter tuning compression ratio for Conv2D layer 1: 55.18
The hyperparameter tuning compression ratio for Conv2D layer 2: 1.43
The hyperparameter tuning compression ratio for Conv2D layer 3: 0.36


## 3) Now randomly deleting half of the data from the original dataset.

In [37]:
import numpy as np

(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = train_images[..., np.newaxis]
test_images = test_images[..., np.newaxis]

np.random.seed(42)  
train_mask = np.random.rand(len(train_images)) < 0.5
test_mask = np.random.rand(len(test_images)) < 0.5


reduced_train_images = train_images[train_mask]
reduced_train_labels = train_labels[train_mask]
reduced_test_images = test_images[test_mask]
reduced_test_labels = test_labels[test_mask]


print("Reduced training set shape:", reduced_train_images.shape)
print("Reduced training labels shape:", reduced_train_labels.shape)
print("Reduced test set shape:", reduced_test_images.shape)
print("Reduced test labels shape:", reduced_test_labels.shape)


Reduced training set shape: (29991, 28, 28, 1)
Reduced training labels shape: (29991,)
Reduced test set shape: (4965, 28, 28, 1)
Reduced test labels shape: (4965,)


In [38]:
reduced_train_images, reduced_test_images = reduced_train_images / 255.0, reduced_test_images / 255.0
reduced_train_images = reduced_train_images[..., tf.newaxis]
reduced_test_images = reduced_test_images[..., tf.newaxis]

In [39]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

In [40]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [41]:
history = model.fit(reduced_train_images, reduced_train_labels, epochs=10, 
                    validation_data=(reduced_test_images, reduced_test_labels))

Epoch 1/10
938/938 [==============================] - 7s 8ms/step - loss: 1.0369 - accuracy: 0.6424 - val_loss: 0.4357 - val_accuracy: 0.8649
Epoch 2/10
938/938 [==============================] - 6s 7ms/step - loss: 0.3953 - accuracy: 0.8783 - val_loss: 0.3358 - val_accuracy: 0.9003
Epoch 3/10
938/938 [==============================] - 7s 7ms/step - loss: 0.3029 - accuracy: 0.9048 - val_loss: 0.2869 - val_accuracy: 0.9168
Epoch 4/10
938/938 [==============================] - 6s 6ms/step - loss: 0.2354 - accuracy: 0.9269 - val_loss: 0.1980 - val_accuracy: 0.9404
Epoch 5/10
938/938 [==============================] - 7s 8ms/step - loss: 0.1833 - accuracy: 0.9425 - val_loss: 0.1586 - val_accuracy: 0.9513
Epoch 6/10
938/938 [==============================] - 12s 13ms/step - loss: 0.1513 - accuracy: 0.9530 - val_loss: 0.1460 - val_accuracy: 0.9543
Epoch 7/10
938/938 [==============================] - 11s 12ms/step - loss: 0.1288 - accuracy: 0.9597 - val_loss: 0.1162 - val_accuracy: 0.9637
Ep

In [42]:
reduced_test_loss, reduced_test_acc = model.evaluate(reduced_test_images,  reduced_test_labels, verbose=2)
print('\nTest accuracy for reduced dataset:', reduced_test_acc)

156/156 - 0s - loss: 0.0789 - accuracy: 0.9754 - 318ms/epoch - 2ms/step

Test accuracy for reduced dataset: 0.9754279851913452


In [57]:
input_dim = (28, 28, 1)

X_train=reduced_train_images.reshape(reduced_train_images.shape[0], reduced_train_images.shape[1], reduced_train_images.shape[2], 1)
X_train=X_train / 255.0
X_test = reduced_test_images.reshape(reduced_test_images.shape[0], reduced_test_images.shape[1], reduced_test_images.shape[2], 1)
X_test=X_test/255.0

t_size = np.prod(input_dim)
t_params = model.count_params()
compression_ratio = t_params / t_size
print("The reduced dataset compression ratio is:", compression_ratio)
print()

input_size = np.prod(input_dim)
output_size = np.prod(model.layers[-8].output_shape[1:])  
t_params = model.count_params()
compression_ratio = output_size / input_size
print("The reduced dataset compression ratio after last layer is:", compression_ratio)
print()

compression_layer_ratios = []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        input_size = np.prod(layer.input_shape[1:])
        output_size = np.prod(layer.output_shape[1:])
        compression_ratio_layer = output_size / input_size
        compression_layer_ratios.append(compression_ratio_layer)

total_cr = np.prod(compression_ratio_layer)
print("The reduced dataset total compression ratio is:", total_cr)
print(compression_layer_ratios)
print()
for i, ratio in enumerate(compression_layer_ratios):
    print(f"The reduced dataset compression ratio for Conv2D layer {i+1}: {ratio:.2f}")

The reduced dataset compression ratio is: 119.03316326530613

The reduced dataset compression ratio after last layer is: 27.591836734693878

The reduced dataset total compression ratio is: 0.36
[27.591836734693878, 1.4319526627218935, 0.36]

The reduced dataset compression ratio for Conv2D layer 1: 27.59
The reduced dataset compression ratio for Conv2D layer 2: 1.43
The reduced dataset compression ratio for Conv2D layer 3: 0.36


## 4) Now deleting half the data from the original dataset randomly and hyperparameter tuning

In [46]:
(train_images, train_labels), (test_images, test_labels) = datasets.mnist.load_data()

train_images = train_images / 255.0
test_images = test_images / 255.0

train_images = train_images[..., np.newaxis]
test_images = test_images[..., np.newaxis]

np.random.seed(1)  
train_mask = np.random.rand(len(train_images)) < 0.5
test_mask = np.random.rand(len(test_images)) < 0.5

reduced_train_images = train_images[train_mask]
reduced_train_labels = train_labels[train_mask]
reduced_test_images = test_images[test_mask]
reduced_test_labels = test_labels[test_mask]

print("Reduced training set shape:", reduced_train_images.shape)
print("Reduced training labels shape:", reduced_train_labels.shape)
print("Reduced test set shape:", reduced_test_images.shape)
print("Reduced test labels shape:", reduced_test_labels.shape)

Reduced training set shape: (29949, 28, 28, 1)
Reduced training labels shape: (29949,)
Reduced test set shape: (4988, 28, 28, 1)
Reduced test labels shape: (4988,)


In [47]:
reduced_train_images, reduced_test_images = reduced_train_images / 255.0, reduced_test_images / 255.0

reduced_train_images = reduced_train_images[..., tf.newaxis]
reduced_test_images = reduced_test_images[..., tf.newaxis]

In [48]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(10)
])

In [49]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [50]:
history = model.fit(reduced_train_images, reduced_train_labels, epochs=10, 
                    validation_data=(reduced_test_images, reduced_test_labels))

Epoch 1/10
936/936 [==============================] - 7s 7ms/step - loss: 0.9920 - accuracy: 0.6560 - val_loss: 0.4051 - val_accuracy: 0.8699
Epoch 2/10
936/936 [==============================] - 6s 7ms/step - loss: 0.3412 - accuracy: 0.8931 - val_loss: 0.2557 - val_accuracy: 0.9238
Epoch 3/10
936/936 [==============================] - 7s 7ms/step - loss: 0.2401 - accuracy: 0.9262 - val_loss: 0.1907 - val_accuracy: 0.9419
Epoch 4/10
936/936 [==============================] - 6s 7ms/step - loss: 0.1780 - accuracy: 0.9444 - val_loss: 0.1472 - val_accuracy: 0.9567
Epoch 5/10
936/936 [==============================] - 7s 7ms/step - loss: 0.1405 - accuracy: 0.9552 - val_loss: 0.1310 - val_accuracy: 0.9593
Epoch 6/10
936/936 [==============================] - 8s 9ms/step - loss: 0.1185 - accuracy: 0.9625 - val_loss: 0.1053 - val_accuracy: 0.9685
Epoch 7/10
936/936 [==============================] - 12s 13ms/step - loss: 0.1001 - accuracy: 0.9675 - val_loss: 0.1021 - val_accuracy: 0.9675
Epoc

In [51]:
reduced_test_loss, reduced_test_acc = model.evaluate(reduced_test_images,  reduced_test_labels, verbose=2)
print('\nTest accuracy for reduced dataset:', reduced_test_acc)

156/156 - 0s - loss: 0.0820 - accuracy: 0.9747 - 323ms/epoch - 2ms/step

Test accuracy for reduced dataset: 0.9747393727302551


In [58]:
input_dim = (28, 28, 1)

X_train=reduced_train_images.reshape(reduced_train_images.shape[0], reduced_train_images.shape[1], reduced_train_images.shape[2], 1)
X_train=X_train / 255.0
X_test = reduced_test_images.reshape(reduced_test_images.shape[0], reduced_test_images.shape[1], reduced_test_images.shape[2], 1)
X_test=X_test/255.0

t_size = np.prod(input_dim)
t_params = model.count_params()
compression_ratio = t_params / t_size
print("The reduced dataset with hyperparameter tuning compression ratio is:", compression_ratio)
print()

input_size = np.prod(input_dim)
output_size = np.prod(model.layers[-8].output_shape[1:])  
t_params = model.count_params()
compression_ratio = output_size / input_size
print("The reduced dataset with hyperparameter tuning compression ratio is:", compression_ratio)
print()

compression_layer_ratios = []
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D):
        input_size = np.prod(layer.input_shape[1:])
        output_size = np.prod(layer.output_shape[1:])
        compression_ratio_layer = output_size / input_size
        compression_layer_ratios.append(compression_ratio_layer)

total_cr = np.prod(compression_ratio_layer)
print("The reduced dataset with hyperparameter tuning total compression ratio is:", total_cr)
print(compression_layer_ratios)
print()
for i, ratio in enumerate(compression_layer_ratios):
    print(f"The reduced dataset with hyperparameter tuning compression ratio for Conv2D layer {i+1}: {ratio:.2f}")

The reduced dataset with hyperparameter tuning compression ratio is: 119.03316326530613

The reduced dataset with hyperparameter tuning compression ratio is: 27.591836734693878

The reduced dataset with hyperparameter tuning total compression ratio is: 0.36
[27.591836734693878, 1.4319526627218935, 0.36]

The reduced dataset with hyperparameter tuning compression ratio for Conv2D layer 1: 27.59
The reduced dataset with hyperparameter tuning compression ratio for Conv2D layer 2: 1.43
The reduced dataset with hyperparameter tuning compression ratio for Conv2D layer 3: 0.36
